In [ ]:
from qdrant_client import QdrantClient, models
import json 


In [ ]:
qd_client = QdrantClient("http://localhost:6333")

In [ ]:
from fastembed import TextEmbedding

In [ ]:
tables=[]
texts= []

In [ ]:
with open('../data/summarized_tables.json', 'r' , encoding="utf-8") as f1:
    tables = json.load(f1)

with open('../data/summarized_texts.json', 'r' , encoding="utf-8") as f2:
    texts = json.load(f2)


In [ ]:
# configurations

model_handle = "jinaai/jina-embeddings-v2-small-en"
EMBEDDING_DIMENSIONALITY = 512

In [ ]:
collection_name = "stardew-vector-search"

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

In [ ]:
table_points = []

for i , table in enumerate(tables):

    text_to_embedd = f"Page title: {table['page_title']}. Section title: {table['section_title']}. Table summary: {table['summary']}"
    
    point = models.PointStruct(
        vector = models.Document(text=text_to_embedd, model=model_handle), 
        id=i,
        payload=table
    )
    table_points.append(point)

In [ ]:
text_points = []

for i , text in enumerate(texts):
    text_to_embedd = f"Page title: {text['page_title']}. Section title: {text['section_title']}. text: {text['text']}"
    text["type"] ="text"
    point = models.PointStruct(
        vector = models.Document(text=text_to_embedd, model=model_handle), 
        id=i,
        payload=text
    )
    text_points.append(point)

    

In [ ]:
def batch_upsert(qd_client, collection_name, points, batch_size=500):
    total = len(points)
    for i in range(0, total, batch_size):
        batch = points[i : i + batch_size]
        qd_client.upsert(collection_name=collection_name, points=batch)
        print(f"✅ Upserted {min(i+batch_size, total)}/{total}")

In [ ]:
batch_upsert(qd_client=qd_client,collection_name=collection_name,points=text_points,batch_size=500)

In [ ]:
for i in range(12):
    print("epoch ", i+1)
    qd_client.upsert(
        collection_name=collection_name,
        points=table_points[i*1000:(i+1)*1000]
    )



In [ ]:
qd_client.upsert(
    collection_name=collection_name,
    points=table_points[12000:12010]
)

In [ ]:
def vector_search(query, limit=5):
    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally
            text=query,
            model=model_handle 
        ),
        limit=limit,
        with_payload=True #to get metadata in the results
    )

    return results


In [ ]:
question = "what crops to plant in fall?"

query_points = vector_search(query=question)

results = []
for qPoint in query_points.points:
    results.append(qPoint.payload)

results